In [44]:
import numpy as np

# Flip bit order in a byte (LSB <-> MSB)
def swap_bits(byte):
    return int(f"{byte:08b}"[::-1], 2)

# Whiten/dewhiten a list of bytes using a 7-bit LFSR.
def BLE_whitening(data, lfsr=0x01, polynomial=0x11):
    # Input data must be LSB first (0x7C = 124 must be formatted as 0011 1110)
    # LFSR default value is 0x01 as it is the default value in the nRF DATAWHITEIV register
    # The polynomial default value is 0x11 = 0b001_0001 -> x⁷ + x⁴ + 1 (x⁷ is omitted)
    output = np.empty_like(data)  # Initialise output array

    for idx, byte, in enumerate(data):
        whitened_byte = 0
        print(f'\n##### INDEX: {idx}, BYTE: {byte:08b} #####\n')
        for bit_pos in range(8):
            print(f'\nbit_pos: {bit_pos}')

            # XOR the current data bit with LFSR MSB
            lfsr_msb = (lfsr & 0x40) >> 6  # LFSR is 7-bit, so MSB is at position 6
            data_bit = (byte >> bit_pos) & 1  # Extract current bit
            whitened_bit = data_bit ^ lfsr_msb  # XOR
            print(f'data_bit: {data_bit}, lfsr_msb: {lfsr_msb}, whitened_bit: {whitened_bit}')
            
            # Update the whitened byte
            whitened_byte |= (whitened_bit << (bit_pos))
            print(f'whitened_byte: {whitened_byte:08b}')

            # Update LFSR
            lfsr = (lfsr << 1) & 0x7F  # 0x7F mask to keep ksfr within 7 bits
            if lfsr_msb:  # If MSB is 1 (before shifting), apply feedback
                lfsr = lfsr ^ polynomial  # XOR with predefined polynomial

            print(f'lfsr: {lfsr:07b}')

        output[idx] = whitened_byte

    return output

# Example usage:
data = [0b00000010, 0b01010011, 0b01111101, 0b11100011, 0b10011000]  # Example received SDR data
data = [swap_bits(byte) for byte in data]
whitened = BLE_whitening(data)
print([f"0b{b:08b}" for b in whitened])  # Output in binary
print([f"{b}" for b in whitened])  # Output in decimal


##### INDEX: 0, BYTE: 01000000 #####


bit_pos: 0
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0000010

bit_pos: 1
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0000100

bit_pos: 2
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0001000

bit_pos: 3
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0010000

bit_pos: 4
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0100000

bit_pos: 5
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 1000000

bit_pos: 6
data_bit: 1, lfsr_msb: 1, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0010001

bit_pos: 7
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0100010

##### INDEX: 1, BYTE: 11001010 #####


bit_pos: 0
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 1000100

bit_pos: 1
data_bit: 1, lfsr_msb: 1, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0011001

b